<a href="https://colab.research.google.com/github/aubricot/computer_vision_with_eol_images/blob/master/classification_for_image_tagging/flowers/classification_display_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Display & export results from trained classification models for images
---
*Last Updated 3 Aug 2020*   
Display classification results from trained classification models on images and verify that they are as expected (or to further fine tune the classification model accordingly, ex: adjust hyperparameters from drop-down menus and re-train). Export resulting classifications to file for use as EOLv3 image tags.

**Notes**
* Change filepaths or information using the form fields to the right of code blocks (also noted in code with 'TO DO')
* Make sure to set the runtime to GPU Hardware Accelerator with a High Ram Runtime Shape (Runtime -> Change runtime type) 

## Imports   
---

In [ ]:
# Mount google drive to import/export files
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# For working with data and reading/displaying images
import itertools
import os
import numpy as np
import pandas as pd
from skimage.transform import resize
import matplotlib.pyplot as plt
import time

# For image classification and training
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

## Run images from URL through trained classifer without exporting results
---

### Define functions and select model

In [ ]:
# Load trained model from path
model_selection = ("06_inception", 224) #@param ["(\"25_fromscratch\", 150)", "(\"20_fromscratch\", 150)", "(\"02_mobilenetssd\", 224)", "(\"06_inception\", 224)"] {type:"raw"}
TRAIN_SESS, pixels = model_selection
saved_model_path = '/content/drive/My Drive/summer20/classification/saved_models/' + TRAIN_SESS
flower_model = tf.keras.models.load_model(saved_model_path)
dataset_labels = ['Branch', 'Entire', 'Flower', 'Fruit', 'Leaf', 'Stem']

# Load in image from URL
# Modified from https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/saved_model.ipynb#scrollTo=JhVecdzJTsKE
def image_from_url(url, fn):
  file = tf.keras.utils.get_file(fn, url) # Filename doesn't matter
  disp_img = tf.keras.preprocessing.image.load_img(file)
  img = tf.keras.preprocessing.image.load_img(file, target_size=[pixels, pixels])
  x = tf.keras.preprocessing.image.img_to_array(img)
  #x = tf.keras.applications.mobilenet_v2.preprocess_input(
    #x[tf.newaxis,...])
  x = tf.keras.applications.inception_v3.preprocess_input(
    x[tf.newaxis,...])
  return x, disp_img

# Run image through classifier
def run_model(x):
  infer = flower_model.signatures["serving_default"]
  label_id = infer(tf.constant(x))[flower_model.output_names[0]]
  label = dataset_labels[np.argmax(label_id)]
  confidence = format(np.amax(label_id)*100, '.3f')
  return label, confidence

### Single Image

In [ ]:
# TO DO: Insert image URL using form field to the right
url = "data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBxIQEhUSEBAVFRUWFhgXFRUVFRUVFRUVFRUXFhUXFRcYHSggGBolHRUVITEhJSkrLi4uFx8zODMtNygtLisBCgoKDg0OGhAQGC0dHR0tLS0rLS0tKy0rLS0tLS0tKysrLS0tLS0uLSstLS0rKy0tKy0rLS0tLSstNystLS0tK//AABEIAQoAvgMBIgACEQEDEQH/xAAcAAABBQEBAQAAAAAAAAAAAAAEAAEDBQYCBwj/xABAEAABAwIEAgYGCAMJAQAAAAABAAIRAwQFEiExQVEGImFxgaETMpGxwdEHI0JScpLh8BRiohUkM0OCssLS8VP/xAAZAQEBAQEBAQAAAAAAAAAAAAAAAQIDBAX/xAAqEQEBAAICAgEBBwUBAAAAAAAAAQIRAyESMUETBCJRYXGhsUKR0fDxMv/aAAwDAQACEQMRAD8AFTpJLzPQSSdJEJJJJAkkk6BJJJFFJJRG6pj7bfzBdCu37w9oQSJJAynQMknSQMknSQMknSRDJJ0kHKSdJBykknQMnSToGSTpIGUVxctpiXH5k8gFDiV6KLZO52CytxiZcS4936AJoWlxjVVxIphre12p+Q81zTtq1TV5c8cg4eQJjyVdYZnmSCO8D3n5K8bTkSYkcQMvmCqp6L2M3plsc2gjxyk+5FelBEQ2OcQPJBNuX09CQ4fzQSO4wo6twTwjtEg/qg7rl7T1HfMdxC7oYq5mlQZhz0zeWhVdVuHNO2h4jaez5Lr0gdBEToTGk/NBo7a5ZUEscD7x3jgplmabXDr0zrG43I/5BXdjeh/Vdo8DUc+0KAxMnSQJJJOiGTLpMgZKE6SDlJOkopk6SdAyZxhdIDFa+Vp14KjL4/eGrULW7N0nh4IWhRiDBJ9p8OQR2FYSasuJIBOgWqssCaNv1WcuSTp2w4bl2z1lnBGmnD9VetDnNkbDhJHnEBXNv0epHU78xofJFu6PUiN3eJn3rE5Y6XgrF1HA6kGfH4boH0jSdjxjSfet+7o9R4yVB/YlFv8Alz3lPqrPs7BCo8af0uXFcmdW5fd5L0CtZ0yINNvsVZd2TPVygJOVLwaZi0qmYzeEgGeztR1NjhqZ3lpG47vkoLmwyE6dxH71U1mXEZS7SNzwOw137NV19vNZpeWtXM2TvxUqrrGrDo56HsIVmohkk6SBky6SVRylCdJBykknUUkk6SBll+kdY+kyDlt3xC1KyXSPq1i6ODI8x8VRocFoZabRGyvrdUeD1JpjuVzbleTL2+nx+lrRcFKXIGm88ESHGNVGtGe480PVKINOVC+j2IoclV9+3RH1aZCDrO4O2VjnkoYl2uo2PMIyjagOIA194jlyIQl9Yua7PSI7W/eHJFWF3nDXcdvZqPiPAL043p4eTHVM+kGiQNfMhGtMgFQ1XAujgeXPX9PJTUxpHJaczpJ0lAyZdJkQySdJUcJ0k6imTpJ0DKrx6yFRhPFuU+AOqtVHctBaQdjoVRTdHK8gjl5LVWolZPCGehdVBE9YFscQRw9hRV5jZp8Y7B89z4ALz5Y/ee3jz1i2DKrGmC4SiWPbrBnVeT3mLVKhBa0DrQHdY6xMSN9JWk6PXVbq03vAzklr4BPVHWaBtPGTyOiXHTePJtqr3EW0hJWTxPpS5p9drZ2a0Fzj3xt7UV0polrJzvdLgIdlgTMRA5wPFVWC4YHw9w0PrGdY4Nby7Sk0ZW26cjpK87gg/hPulH08VlmZxBbuXCSfy6+S4PRmnnLzyAAgCAJjbc66ndSW+FCm0huxOgPCSrbPhiTL5cuvcwHo6b3zrtlA780Qq2rTewVDm9HmIIDesROjoJgA7cDur15y7eGn6oC6p5gQdQRr3jY96uOXbPJhbAGGUajTmLyZ0E8YgDx0ROEXz/SvpVhDpkRsQdlDdXzGOZSfIIaHSOBkj5qZ7AalOqDrsT36fvvS5dt4cUuH6r1JOEl1eAyZdJIOUk6So4TpJ1FMnSToGUV0YY4jkpkz2yISrPfbM21PK70gL+sJyuJgTyG3JTHDBXcHP2HDn3qwFIfl4dm0e5WNu0mMob4hcLXunHq9AH4fTLsxaC+ABA1007hsjLW2aarHZdWB2vMu08Y1/YR9KxJ9aAOz5bLsUw10D2rNrpMHOK0QWweKAtLVzDDQCDz39qtrpkt2VZb18ro4BJ6ayk2K/hHncR3CfOVI61IHzRYuxG6hrXEhRZiqq9uQFVuKt7qvwVQ8arTnkpuktrncHt/xGjYAyWcNuWqOwSk402B4+1I7kUQA8OgExExrH796sLOgG6+xak2zln4YCITp0l2fOcpLpMgZMukyDhOknRSSTpIEknSQB3NODPOR48ERZ1IAhdvYHCCggchI5e7guWePy9nByb6vwtzcwN1FVxJlIh1TY6TGkoJjiXCdgi6bQ4QQCFzkem5GvceYGkg7LP0sTqPJ+rAadjmknvEaI7Fej1Ors2CDw8l3bYcKYAPBammLbaKt5yAO3SzEECZB2Q91esaPWAjtCVjWFbKW7ZgZjcAqWHl2kutECAjsRPWAQrgkTJDZya2Uxl1d293uV0qvDY9Iecae0K1XbD08XLb5GSTpLTkZJOkg5STpII06dJFJJOkgSSdJAkJfU9nctD3FGJnNkQUs2uN1dg6Ore0IZ1SuPUayJ4uIjwATuJpPg7e8cCjDqNF5/T342ZdoaVvcP1NRo7pj9U1TCgdatYu5gQ2fHUpPoVHGA7Ts0UjMIJ1c7+omVfJ030Fbh9GQGsBjnJ9pKsaTA3UcF3Ttw0KO5qgaBZt2z6CVnS/x+ChrOhO+p1pQr6u5nQKse02HVQ2oJ4gju2V2COCzjactk8du5ddDrN4r1WkktDRHjMe5ez6VxwleHPkmWd00SZS1aRaYK4XNlyknSQMmXSZBGnSTopJJ0kCSTpIEknToILq3FQRseB5IKlmp6PHcRsfkrRQ3lD0lNzf5Se6BM+SzljtvDkuLgVxz/VPVvxssjUvqrRBE9uxQVziD2tkmeAErlMHt+r02T78bBB3N4AsK7EbguAB1cdIEK89JDAM23rO7eK1cfFz87kPfczoDqfWP3Rx8Urb6z8A2/mjj3Kqt/rDpozzd39nYtBaU9F6uHg/qyeXm5+vHFPlWk6IYdFN1UjWodPwt0HuJ8VS21m6s9tJm7zE8hu53gJXpVvZBjQxogNAA7houvNl8OGE+VPc2QeI48DyVDWolhyu3W4daqsxXDc403Gx+C8+nRlklNWoOb6whRQoGTLpMoI06SdFJJOkgSSdJEJJOumtnQKjlT3JFG2qVHbuGRv8Aq0+KIt7Tidzsqv6Q6pZToU28arPIyfcVvDHtMr0zVxR4x3/NVNa0a4zOgOy0OYHgqrEWsY4Q4SfsTqfDktc3B/Vi3w8+vu5A227Wgv2gQD38Qh6dEuiRDRs3nHF3yV7Z4U58Oq78G7hvzd2qwZhbeS1w8Gu8vac3P5dY+lNa254bq7tKcN10Upt2s1QjKdS7qtt6O7iAXfdHHxiV6XmbnoDYBwfcEaaspnmAeu7xcI/0raNZCjw+xZQpMpUxDWNDQOwCEY1q8WV8rt3k1HBphDVrdGlRV9Gk9igoqtmHNAI4D3KjvcHI1Z7FsXMaUPUt1RgHsLTBEFcrXX2HB41H6LP3eGvYdBIWbDatTpJ1lokkk6BJJ2tJ0AlF29oDq4+CukQUaJdttzVjStg1qmoAbAKWoNgtaR3bUpI7FjvpAfmuban2vfH4WZf+YW4twsf0rtAa7Kp3yvY0cmywk95I8l04v/TOfpjekOJOt2NDPWeYBicoA1McTyWSpy8y6SXaknUnt5lbPpNaNfQeSNWDO08oGvlKxRMNO23AkeI5rtl7co2vQ3GBUmg89Zolh+80cPD3LUVqwaF4xZXD6dVjqfrhwLRzJO3cdl6vSaX+tp2cit43cSmLXVjGw5rdfR9gzWB1bL/Kz/k74e1Zyytc7xTYNyAvT8NthTptY3YCFjmy1Nfi3hO9imhdJJ15nUyGvakZR950ewF3wRRKp61b0lxlG1Nv9To+CCd/BdMO/euGmXdy64eKqE5oKFq2vZKLCQEKK8zCdJOsNEu6dOf3unpsnVT0x8lZE2dojQKdoUTAiKTZMLSCrZkCVI4JbdiGuMSo0/XqDuGp8lBZ2zVmumDNabuTsp7MwPxAV/hF6KzC9rSGyQJ3MbmOCpOmzYtqjvu5X/kcHfArWF1kmXp5900rFjfRDWes8jYN0gT2ujwDljrkaESRoPWAPfrw1Wg6UViC1hIl5LjH3W9VgnxcfaqWrqSM8gu4gwdCMx7pXov5uKx6DYWH1HVX65DoO2JlbqgOsdeO/KAB81luhlSDWJiMjDAjhmbEczAWspUj1WjVxO3NxPzK3j1EbfozhrA41GvzAEtB4bDUdupC1zNFU4Ra+hptbvlAk83cT7ZXWJ4nUoln1QeHTsYII1jXfT3LyZXdd5NRcBOXQqSn0hB3t6wP4Z9yivMehsim4afa0UVaXt0KbC88Nu08AqTAnS2pVO7ne7/1U1e9rVgS4E8gBorjD2FlJjDuTJ8UFpb7DtK7fsuQYIC6edVQ7V0uM0ISpdydNVEYJIlJcndYjQqloz2qSkNAoaRlp/e6mpnQLaOpXVWkXtLQ4tJ+0Nx3Lhg1RDCqilucDqO2rud+IlWOG4DSDAHtl3EzxlG00RbuiP3qoDLKk1jcjRAGwVZ0poZ7eq3nTcP6SrWiVBi1PNTcOYPuSe1rwjpdVm4ERAY0dupzae0eaGo21SoTkp54eJytzAHWNRs3yQVw5z3y5pLpg77iB8I8VvugNSsy0ujRpuJ9LTa4MJbUawtcHFhGoc2QfArvcvlxkU3RSk5lRzX0y2QDJDhOR5PHvXpnRG1FSrnJEMGmv2j8vkiujlxWZRqCq2q4UnllNzy59WsGmHP5wTMdgR7MWa4gOpVRJAGamY6xAEnhqVm8l1pqY9tLTCgxGiX03R6w6zfxN1Ht1His+7FqTQ0vpvAc0H/DJgkuGUxMEZfMKT+0qWcMymSGkHKMvXc5oE8wWmRwXJva4oVg9rXDYgH2qbKDuAe9UmFXhNarRDQG04MjTVwBiPariVR1IA2AQufr5uQ070q1VD0S2ZNRo7JErNyk91dUSKhJkojNOqgeARISpOVRBit1lEDipsMoACTyVPitWatNvN3kFetdlaEHn6jpmRK7fsVE0Q7sPvWY1RNF2sc2hTMOgQbzGQ9kIlh0C0ynZsnpP1UVd0Bd0lQZTOhUzFANgFO1QHUHSFJciWlCWj0aRIIUV824rSDLiqwz1arxw2zn4LSdDOmjMNZUa6i+p6RwOjg3LAjjuqzpfTLL+uA6PrMw7JaDKqHEgDVujSeGg1kHTf8ARd9bjj6r2zol02OJPc2hZ1AGAZnuewNE7DtO/sWtyVP/AJ+xwVJ9G+A/wVkxrmxUf9ZU55nbNPcIHgtXK4326yK76wf5LvzM/wCyqqvSSkx5YW1C4GCGtDtfAozpDiZaPRUvXdppwB+KjwjDG0GbS86uPbyQPgLCX1qpa5vpHgtDhDsrWNAkcNcybpT0kpWNPM8y4+q0c0e6plBPISvD+kmOenu3vqNFSmCWZCY6s6lrvsukaOHIDUSDmy5ZeM/W/wC/jWp1NtTe4hdXAL7iv/DsDyw0xlaR90+kd1dZEQNee5FL/F22c/32tGVn+bWHWDiahBH8oEdp4bLLYjiD6zpe4wAA1skhjQIAEk8BvudyhZVw+yT3b3+X+fdS8j1HDcXr2wFVtc1qOuZrnNqPa1roL2vb6zdtHDnqCvQrK5bUYHtOhEjxXztYXGVwa4n0Zc3O2TBAI3A328l7T0Jv2VbdwpulrHFoMZdOAA4AAhYvHeLOT4v9v+/y1vyjq+q/3umOXxV9iNbKAsrcVJvmDtaPILV1iM0kTwXRhhrp8COaUyJ7j80PcPl8ctPHiprc6Ry9xUi13V9Xud70VR2CEbqCOY8wimHqhVHFd0uaPFFUd0BTdLyeQRtsZQFzqpmlC0yiGFBPb7Kxtnyq22KLoug96ivG/pRtDTxF5yyH02u8Yc0/7Z8FD9H2B/xd4wPZ1KQFV8zBggs/MY05ArRfTRajPb1uYfTMc9HNn2OWn+jDBBbWgqEHPWioZ3DY6jfZr/qXXy+6xr7zatQWLYiKLdNXHRo7eakubkU2lztgqTC6ZuKhrVPVb6o4T+i5OgnDLLL9ZU1e7XXhPxVgXJi5cytIhxE/VPj7pXz4+m+MxaYnV0GJPavoW5ZmYQvOMQtjUp17SoWh7XZqbnEgvzHNSnTb1mzrqNpIXKcn0+S34sn7W7/mLrcRdDre3qWrGVMhqF73AHKTlqPZbmZ75XFHAqFRrJYwvNyyvUDS2W29Wo/LSyjVjMgYdgOv2LBVKcGCNU9Ku5pJa9zZ3yuI9sLpfs+Xlcsc/bPn+MaXpTh9OkxhZQ9EQ9rJBf8AWgW9J73EOMeu8iRHJaj6J3kUq44F/nlbPwWCwylUuqrRUe94bq4ucXZWDUgFxgTEDvXqnRKybRpuyNyhzi4CSd47/shh8VnK+Mx47d321O95Iqz4v2/ib/tBW19Hm3WFricSpjnlPgGmfctbUuHToVplgbd0uPbqjKboeO0KtpPhyspB3RXQJBI4gyP37VJRdouX8He3uTNO6Dq2+0UXQdAQdtspw/QIg2kVOxyFpFStcgNtyiAUJbHREZtUVX9K8GF9TpUzsKzHO/AJzjxaSPFaJkNHIBC0N0Li95AyN3KgCxO4dXeKbNp/9KvKVMU2BjdggcHs8gzu9Y+QRtR6sClMSoy5T+j7VQmlZrpRgfpodTIbVaHZHOaHNhwhzagIMtIPhvzWm9HHFPTpgzx4eHFYzx8osunkuKW1KoQL6kbd/qZyXAOLQ2HCrBY5hJIkyePNVL8Ctg3N/E6ZGuGZzGznboDOg1gHlB1XsV3h0zDiAd26Fv5XAhA0cAptOYspA82UabXe2CuM+rj1P56/edfu1fGsV0ewQPJZSaTRc/MHOBaXBoAgHd7JE59gHQJJW+NMMAaOHnzKKpUGsBIG+5JknvJ9yCruXTDCy+WXtm34jP1HRidM86PnmcPctQ1ZPEjF5bHm1w9jm/8AZbFtOF0Zeb1BxVjbO0QT9kVY7BFFgqMcVIVGeKDq3KVN65t1GNz3oLWidF2HKGhspEQfbOU7DqhLbgpxuiiLq7FJhcfAcyg8LomqfSP8ED0mOre74q9w8fVt7goopzkM96mchXKxHbXahFoClujilHNZ65zSuK6Zqgm9Iea5NU9i4K5cUEjq5jUDzUBrAfZHn801VQVEGa6W1IfRrNABpkjSdnFvxaPatHh12X0w6ZlZvpWOofw/FWPRU/3Vnefeqj//2Q==" #@param {type:"string"}
fn = "angiosperm_image.jpg"
img, disp_img = image_from_url(url, fn)
label, confidence = run_model(img)

# Plot and show cropping boxes on images
_, ax = plt.subplots(figsize=(10, 10))
ax.imshow(disp_img)
plt.axis('off')
plt.title("Prediction: {}, Confidence: {}%".format(label, confidence))

### Multiple images

In [ ]:
# TO DO: Enter URLs here
url1 = 'https://content.eol.org/data/media/80/ce/d7/542.6789991664.jpg' #@param {type:"string"}
url2 = 'https://content.eol.org/data/media/80/d7/17/542.6872776289.jpg' #@param {type:"string"}
url3 = 'https://content.eol.org/data/media/81/12/42/542.7670415710.260x190.jpg' #@param {type:"string"}
urls = [url1, url2, url3]
print(urls)

# Classify images from URL and display results
for im_num, url in enumerate(urls, start=1):
  fn = str(im_num) + '.jpg'
  img, disp_img = image_from_url(url, fn)
  # Record inference time
  start_time = time.time()
  # Classify image
  label, confidence = run_model(img)
  end_time = time.time()
  # Display progress message after each image
  print('Classification complete in {} of {} images'.format(im_num, len(urls)))

  # Plot and show cropping boxes on images
  _, ax = plt.subplots(figsize=(10, 10))
  ax.imshow(disp_img)
  plt.axis('off')
  plt.title("{}) Prediction: {}, Confidence: {}%, Inference Time: {}".format(im_num, label, confidence, format(end_time-start_time, '.2f')))

### Run images from EOL image bundles

In [ ]:
# Read in EOL image bundle dataframe
# TO DO: Type in image bundle address using form field to right
bundle = 'https://editors.eol.org/other_files/bundle_images/files/images_for_Angiosperms_20K_breakdown_000031.txt' #@param {type:"string"}
df = pd.read_csv(bundle, sep='\t', header=0)
df.head()

In [ ]:
# Optional: Run inference for taxon-specific images only
# TO DO: Type in the taxon you'd like to inspect results for using form field to right
taxon = "" #@param {type:"string"}
df = df.loc[df.ancestry.str.contains(taxon, case=False, na=False)]
print(df.head())

In [ ]:
# Set number of seconds to timeout if image url taking too long to open
import socket
socket.setdefaulttimeout(10)

# TO DO: Set start and end rows to run inference for from EOL image bundle using form field to right
start =  0#@param {type:"integer"}
end = 50 #@param {type:"integer"}

# Loop through EOL image bundle to classify images and generate tags
for i, row in df.iloc[start:end].iterrows():
  try:
    # Get url from image bundle
    url = df['eolMediaURL'][i]
    # Read in image from url
    fn = str(i) + '.jpg'
    img, disp_img = image_from_url(url, fn)
    # Record inference time
    start_time = time.time()
    # Run inference/Classify image
    label, confidence = run_model(img)
    end_time = time.time()
    # Display progress message after each image
    print('Classification complete in {} of {} images'.format(i, len(df)))

    # Show classification results for images
    _, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(disp_img)
    plt.axis('off')
    plt.title("{}) Prediction: {}, Confidence: {}%, Inference Time: {}".format(i, label, confidence, format(end_time-start_time, '.3f')))

    # Export tagging results to tsv
    # Define variables for export
    identifier = df['identifier'][i]
    dataObjectVersionID = df['dataObjectVersionID'][i]
    ancestry = df['ancestry'][i]
    with open(tags_fpath, 'a') as out_file:
      tsv_writer = csv.writer(out_file, delimiter='\t')
      tsv_writer.writerow([url, identifier, dataObjectVersionID, ancestry, label])

  except:
    print('Check if URL from {} is valid'.format(url))

## Run images from URL through trained classifer & export tagging results to tsv
---

### Define functions and select model

In [ ]:
import csv

# Load trained model from path
model_selection = ("06_inception", 224) #@param ["(\"25_fromscratch\", 150)", "(\"20_fromscratch\", 150)", "(\"02_mobilenetssd\", 224)", "(\"06_inception\", 224)"] {type:"raw"}
TRAIN_SESS, pixels = model_selection
saved_model_path = '/content/drive/My Drive/summer20/classification/saved_models/' + TRAIN_SESS
flower_model = tf.keras.models.load_model(saved_model_path)
dataset_labels = ['Branch', 'Entire', 'Flower', 'Fruit', 'Leaf', 'Stem']

# Write header row of output crops file
# TO DO: Change file name for each bundle/run abcd if doing 4 batches using form field to right
tags_file = 'angiosperm_tags_20k_a' #@param {type:"string"}
tags_fpath = '/content/drive/My Drive/summer20/classification/results/' + tags_file + '.tsv'
with open(tags_fpath, 'a') as out_file:
                  tsv_writer = csv.writer(out_file, delimiter='\t')
                  tsv_writer.writerow(["eolMediaURL", "identifier", "dataObjectVersionID", "ancestry", "tag"])

# Load in image from URL
# Modified from https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/saved_model.ipynb#scrollTo=JhVecdzJTsKE
def image_from_url(url, fn):
  file = tf.keras.utils.get_file(fn, url) # Filename doesn't matter
  disp_img = tf.keras.preprocessing.image.load_img(file)
  img = tf.keras.preprocessing.image.load_img(file, target_size=[pixels, pixels])
  x = tf.keras.preprocessing.image.img_to_array(img)
  x = tf.keras.applications.inception_v3.preprocess_input(
    x[tf.newaxis,...]) # tried using mobilenet_v2 for mobilenet model and made no difference, keep same for all
  return x, disp_img

# Run image through classifier
def run_model(x):
  infer = flower_model.signatures["serving_default"]
  label_id = infer(tf.constant(x))[flower_model.output_names[0]]
  label = dataset_labels[np.argmax(label_id)]
  confidence = format(np.amax(label_id)*100, '.2f')
  return label, confidence

### Run images from EOL image bundles

In [ ]:
# Read in EOL image bundle dataframe
# TO DO: Type in image bundle address using form field to right
bundle = 'https://editors.eol.org/other_files/bundle_images/files/images_for_Angiosperms_20K_breakdown_000031.txt' #@param {type:"string"}
df = pd.read_csv(bundle, sep='\t', header=0)
df.head()

In [ ]:
# Set number of seconds to timeout if image url taking too long to open
import socket
socket.setdefaulttimeout(10)

# TO DO: Set start and end rows to run inference for from EOL image bundle using form field to right
start =  200#@param {type:"integer"}
end = 250 #@param {type:"integer"}

# Loop through EOL image bundle to classify images and generate tags
for i, row in df.iloc[start:end].iterrows():
  try:
    # Get url from image bundle
    url = df['eolMediaURL'][i]
    # Read in image from url
    fn = str(i) + '.jpg'
    img, disp_img = image_from_url(url, fn)
    # Record inference time
    start_time = time.time()
    # Run inference/Classify image
    label, confidence = run_model(img)
    end_time = time.time()
    # Display progress message after each image
    print('Classification complete in {} of {} images'.format(i, len(df)))

    # Show classification results for images
    _, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(disp_img)
    plt.axis('off')
    plt.title("{}) Prediction: {}, Confidence: {}%, Inference Time: {}".format(i, label, confidence, format(end_time-start_time, '.3f')))

    # Export tagging results to tsv
    # Define variables for export
    identifier = df['identifier'][i]
    dataObjectVersionID = df['dataObjectVersionID'][i]
    ancestry = df['ancestry'][i]
    with open(tags_fpath, 'a') as out_file:
      tsv_writer = csv.writer(out_file, delimiter='\t')
      tsv_writer.writerow([url, identifier, dataObjectVersionID, ancestry, label])

  except:
    print('Check if URL from {} is valid'.format(url))